### Deliverable 1: Preprocessing the Data for a Neural Network

In [198]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [199]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"],inplace=True)

In [200]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [201]:
application_types=application_df["APPLICATION_TYPE"].value_counts()

In [202]:
# Determine which values to replace if counts are less than ...?
replace_application=application_types[application_types <200].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [203]:
# Look at CLASSIFICATION value counts for binning
classification_count=application_df["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [204]:
# Determine which values to replace if counts are less than ..?
replace_class=classification_count[classification_count<1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [205]:
# Dropping noisy data after reviewing the value count and the feature importance
print(application_df["SPECIAL_CONSIDERATIONS"].value_counts())
print(application_df["STATUS"].value_counts())
application_df=application_df[application_df["STATUS"]==1]
print(application_df["STATUS"].value_counts())
application_df=application_df.reset_index()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64
1    34294
0        5
Name: STATUS, dtype: int64
1    34294
Name: STATUS, dtype: int64


In [206]:
application_df.drop(columns=["SPECIAL_CONSIDERATIONS","STATUS"],inplace=True)

In [207]:
# Generate our categorical variable lists
application_cat=application_df.dtypes[application_df.dtypes=="object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [208]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [209]:
# Merge one-hot encoded features and drop the originals
combined_df=application_df.join(encode_df).drop(columns=application_cat)
combined_df.head()

,index,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [210]:
# Split our preprocessed data into our features and target arrays
X=combined_df.drop("IS_SUCCESSFUL",axis=1)
y=combined_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y)

In [211]:
import numpy
numpy.where(numpy.isnan(X_train))

(array([], dtype=int64), array([], dtype=int64))

In [212]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [213]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train_scaled,y_train)

RandomForestClassifier()

In [214]:
importances=rfc.feature_importances_
sorted(zip(importances,X.columns),reverse=True)

[(0.5931274081821771, 'index'),
 (0.10568301093750002, 'ASK_AMT'),
 (0.06226952306825853, 'AFFILIATION_Independent'),
 (0.06223770869074937, 'AFFILIATION_CompanySponsored'),
 (0.015437283189537432, 'ORGANIZATION_Association'),
 (0.014640885654818855, 'APPLICATION_TYPE_T5'),
 (0.01209564105546137, 'APPLICATION_TYPE_T10'),
 (0.00924362325258849, 'CLASSIFICATION_C2000'),
 (0.009047903657117681, 'CLASSIFICATION_Other'),
 (0.008857753595723859, 'ORGANIZATION_Trust'),
 (0.008784739191003043, 'CLASSIFICATION_C2100'),
 (0.008693368457647904, 'APPLICATION_TYPE_T19'),
 (0.008092220178917829, 'APPLICATION_TYPE_T3'),
 (0.007823705286793294, 'APPLICATION_TYPE_T4'),
 (0.007604159499437846, 'CLASSIFICATION_C1000'),
 (0.006534997173271323, 'APPLICATION_TYPE_T6'),
 (0.006190163562469044, 'USE_CASE_ProductDev'),
 (0.006190024682753774, 'USE_CASE_Preservation'),
 (0.005403684056706585, 'CLASSIFICATION_C1200'),
 (0.005224528824605524, 'INCOME_AMT_0'),
 (0.004507158858630795, 'CLASSIFICATION_C3000'),
 (0.0

### Deliverable 2: Compile, Train and Evaluate the Model

In [215]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/Deliverable3",exist_ok=True)
checkpoint_path="checkpoints/Deliverable3/weights.{epoch:02d}.hdf5"

In [222]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features=len(X_train_scaled[0])
hidden_nodes_layer1=80
hidden_nodes_layer2=30
hidden_nodes_layer3=10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,activation="relu",input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_98 (Dense)            (None, 80)                3360      
                                                                 
 dense_99 (Dense)            (None, 30)                2430      
                                                                 
 dense_100 (Dense)           (None, 10)                310       
                                                                 
 dense_101 (Dense)           (None, 1)                 11        
                                                                 
Total params: 6,111
Trainable params: 6,111
Non-trainable params: 0
_________________________________________________________________


In [223]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [230]:
# Callback
Batch_size=59
No_of_steps=int(len(y_train)/Batch_size)
callback=ModelCheckpoint(filepath=checkpoint_path,
                        verbose=1,
                        save_weights_only=True,
                        save_freq=No_of_steps*5)

In [225]:
# Train the model
nn.fit(X_train_scaled,y_train,epochs=50, batch_size=Batch_size, callbacks=[callback])

Epoch 1/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5775 - accuracy: 0.7157
Epoch 2/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5547 - accuracy: 0.7299
Epoch 3/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7302
Epoch 4/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7335
Epoch 5/50
409/436 [===========================>..] - ETA: 0s - loss: 0.5448 - accuracy: 0.7319
Epoch 5: saving model to checkpoints/Deliverable3\weights.05.hdf5
436/436 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7315
Epoch 6/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5434 - accuracy: 0.7359
Epoch 7/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7349
Epoch 8/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5413 - accuracy: 0.7341
Epoch 9/50
436/436 [=============================

In [226]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5511 - accuracy: 0.7347 - 418ms/epoch - 2ms/step
Loss: 0.5510942339897156, Accuracy: 0.7346629500389099


In [227]:
nn.save("AlphabetSoupCharity_Optimization.h5")

In [231]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,rfc.predict(X_test_scaled))

0.702355959878703

In [233]:
from sklearn.linear_model import LogisticRegression
lrmodel=LogisticRegression(solver='lbfgs',max_iter=200)
lrmodel.fit(X_train_scaled,y_train)
accuracy_score(y_test,lrmodel.predict(X_test_scaled))

0.7228831350594822